In [1]:
import glob
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
import numpy as np
from itertools import chain 
from IPython.display import clear_output
from ipywidgets import widgets
from IPython.display import display
from sklearn.model_selection import train_test_split
import os
from sklearn.neural_network import MLPClassifier
import numpy as np
import scipy
%autosave 60

Autosaving every 60 seconds


In [2]:
#display(text)
output = None

In [3]:
def progress(count,total):
        global output
        
        #display(widgets.IntSlider())
        #text = widgets.Text()

        with output:
            print "{0:.2f}".format(count*100.0/total)+" %"
            print '|'+'='*int(count*100.0/total)+'>'+'.'*(100-int(count*100.0/total))+'|'
        with output:
            clear_output(wait=True)

In [4]:
# plt.plot(k.values())
# plt.title("Accel readings in t sec time diff")
# plt.show()

##  DATA PREPROCESSING

In [5]:
path = '/home/rakesh/Desktop/zCap/public_dataset/HMOG/'#100669/'
out_path2 = '/home/rakesh/OUTPUT100/'

In [6]:
def extract(d):
    
    x0,x1,x2 = d[0].min(),d[0].max(),d[0].mean()      
    x3,x4,x5 = d[1].min(),d[1].max(),d[1].mean()
    x6,x7,x8 = d[2].min(),d[2].max(),d[2].mean()
    x9 = d[3].mode()[0]
    l = [x0,x1,x2,x3,x4,x5,x6,x7,x8,x9]
    
    return [float(i) for i in l]

In [7]:
def concat(path,filter_file,idx):
    print filter_file
    df = None
    l = []
    
    for session in glob.glob(path+'*'):
        
        for f in glob.glob(session+'/*'):
        
            if f.split('/')[-1]==filter_file:
                df = pd.read_csv(f,header=None)
                l.append(df)
    df = pd.concat(l)
    df[df.shape[1]] = pd.Series( [idx for _ in range(len(df))], index=df.index)
    return df.sort_values(by=[0]) ## sort by absolute timestamp

In [8]:
def process_acc_gyro(df,gap):
    
    start = df.iloc[0][0]
    db_gyro = []
    db_acc = []
    tmp_gyro = []
    tmp_acc = []
    count = 0
    lengthDF = len(df)
    for index in range(0,lengthDF):

        if index%10000==0:
            progress(index,lengthDF)
            #print "{:.2f} and {}".format((index*1.0)/lengthDF,index),"|",

        row = df.iloc[index].tolist()

        cur_time = row[0]
        #print row 
        #break
        if cur_time<start:
            print "Yes"

        if cur_time>=start and cur_time<= start+gap:
            count+=1
            if row[-1]==0:                      # acc
                tmp_acc.append(row[3:7])

            if row[-1]==1:                      # gyro
                tmp_gyro.append(row[3:7])


        else:
            start = cur_time

            count+=1

            if len(tmp_acc)!=0:            ## ACC
                d = pd.DataFrame(tmp_acc)
                db_acc.append(extract(d))
            else:
                db_acc.append([])



            if len(tmp_gyro)!=0:                ## GYRO
                d = pd.DataFrame(tmp_gyro)

                db_gyro.append(extract(d))
            else:
                db_gyro.append([])


            tmp_gyro = []
            tmp_acc = []

            if row[-1]==0:                      # acc
                tmp_acc.append(row[3:7])

            if row[-1]==1:                      # gyro
                tmp_gyro.append(row[3:7])
            #if tmp.append(row[1:])

    print count 
    return db_acc,db_gyro

In [11]:
def construct(path,out_path,gap):
    global output
    for user in glob.glob(path+'*'):
        user_id = user.split('/')[-1]
        
        if os.path.exists(out_path+str(user_id)+'_acc_gyro.csv'):
            print str(user_id)+'_acc_gyro.csv',"skipped"
            continue
        
        print "User",user_id
        
        acc = concat(user+'/','Accelerometer.csv',idx = 0)  
        gyro = concat(user+'/','Gyroscope.csv',idx=1)
        
        ####################  TRY ######################
#         return acc
        
        
#         break
        ################################################
        
        df = pd.concat([acc, gyro], axis=0).sort_values(by=[0]).reset_index(drop=True)
        
        output = widgets.Output()
        display(output)
        #return df
        db_acc,db_gyro = process_acc_gyro(df,gap)
        
        X1 = pd.DataFrame(db_acc)
        X2 = pd.DataFrame(db_gyro)
        acc_gyro = pd.concat([X1, X2], axis=1)
        print "Saving...",user_id
        
        acc_gyro.to_csv(out_path+str(user_id)+'_acc_gyro.csv',sep = ',',header=False)

construct(path,'/home/rakesh/OUTPUT100ms/',gap=100)
#construct(path,'/home/rakesh/OUTPUT/',gap=10)
#os.system('shutdown -s')

803262_acc_gyro.csv skipped
990622_acc_gyro.csv skipped
472761_acc_gyro.csv skipped
556357_acc_gyro.csv skipped
489146_acc_gyro.csv skipped
User 751131
Accelerometer.csv
Gyroscope.csv


Output()

In [12]:
df

,0,1,2,3,4,5,6,7
0,1397003030460,13684504652000,751131011000001,-1.013944,6.049549,6.032790,0,0
1,1397003030461,13684504682000,751131011000001,0.097433,-0.026573,-0.118508,0,1
2,1397003030505,13684509351000,751131011000001,-0.887650,6.158485,5.743690,0,0
3,1397003030507,13684509351000,751131011000001,1.193631,-0.026573,-0.902553,0,1
4,1397003030509,13684519544000,751131011000001,-0.650624,6.205172,5.633556,0,0
5,1397003030511,13684519544000,751131011000001,-0.563218,-0.739758,0.455095,0,1
6,1397003030512,13684529584000,751131011000001,-0.392649,6.199187,5.640141,0,0
7,1397003030514,13684529615000,751131011000001,-0.664621,0.218384,0.637438,0,1
8,1397003030515,13684539564000,751131011000001,-0.131681,6.161478,5.733514,0,0
9,1397003030517,13684549573000,751131011000001,-0.010175,6.092046,5.889736,0,0


In [ ]:
#acc_gyro.to_csv('100669_acc_gyro.csv',sep = ',',header=False)

In [ ]:
# df = df.interpolate()
# df.shape

In [ ]:
#df.to_csv('100669.csv',sep = ',',header=False)

##   TOuchEvent

In [13]:
path = '/home/rakesh/Desktop/zCap/public_dataset/HMOG/100669/'

filter_file = 'TouchEvent.csv'

In [14]:
def concat(filter_file):
    df = None
    l = []
    
    for session in glob.glob(path+'*'):
    
        for f in glob.glob(session+'/*'):
        
            if f.split('/')[-1]==filter_file:
                df = pd.read_csv(f,header=None)
                l.append(df)
    return pd.concat(l).sort_values(by=[0]) ## sort by absolute timestamp

In [15]:
mdf = concat(filter_file)

In [ ]:
sum(Counter(mdf[2]).values())/113

In [ ]:
#l1 = acc[2].tolist()+gyro[2].tolist()
#sum(Counter(l1).values())/len(Counter(l1))

In [ ]:
mdf = mdf.drop(columns=[1,2])

In [ ]:
start = mdf.iloc[0][0]

gap = 10       # gap of 10ms
count = 0
touch_list = []
tmp = []
length = len(mdf)

output = widgets.Output()
display(output)

for index in range(0,length):
    
    if index%2000==0:
        progress(index,length)
        #print "{:.2f} and {}".format((index*1.0)/length,index),"|",
    
    
    row = mdf.iloc[index].tolist()
    count+=1
    cur_time = row[0]
    if cur_time<start:
        print "Yes"
    
    if cur_time>=start and cur_time<= start+gap:
        
        tmp.append(row[1:])

    else:
        
        start = cur_time
        count+=1
        if len(tmp)!=0:
            
            df = pd.DataFrame(tmp)
            
            x0,x1 = df[0].mode()[0], df[1].mode()[0]    ## pointer count, pointerid
            x2,x3,x4 = df[3].mean(),df[3].max(),df[3].min()      # x-axis
            x5,x6,x7 = df[4].mean(), df[4].max(), df[4].min()  # y-axis
            x8,x9 = df[5].mean(),df[5].max()                ## pressure
            x10,x11 = df[6].mean(),df[6].max()                ##contact size   
            x12 = df[7].mode()[0]                         # phone orientation
            #x13 = len(df)
            tuple_list = [x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12]
            try:
                tuple_list = [float(i) for i in tuple_list]
            except:
                print "\n\n",tuple_list
                break
            #print tuple_list
            touch_list.append(tuple_list)
        tmp = []
        
        tmp.append(row[1:])
print count

In [ ]:
touch = pd.DataFrame(touch_list)
print touch.shape
touch = touch.drop_duplicates()

In [ ]:
touch

## ===============  Merge Files =========================

In [26]:
input_path = '/home/rakesh/OUTPUT/TouchEvent/'
files = 'touchevent.csv'

In [27]:
df1 = pd.read_csv(input_path+'100669_'+files ,header = None).drop(columns=[0]) ## drop index column
df2 = pd.read_csv(input_path+'472761_'+files ,header = None).drop(columns=[0])
df3 = pd.read_csv(input_path+'489146_'+files ,header = None).drop(columns=[0])
df4 = pd.read_csv(input_path+'990622_'+files ,header = None).drop(columns=[0])

In [28]:
df1 = df1.dropna()
df2 = df2.dropna().iloc[0:40000]
df3 = df3.dropna().iloc[0:40000]
df4 = df4.dropna().iloc[0:40000]

In [29]:
df2.shape

(40000, 13)

In [30]:
y1 = [1 for i in range(len(df1))]
y2 = [0 for i in range(len(df2))]
y3 = [0 for i in range(len(df3))]
y4 = [0 for i in range(len(df4))]

y = y1+y2+y3+y4

In [31]:
X = pd.concat([df1,df2,df3,df4], axis=0)
X.shape

(238144, 13)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.25)

In [50]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier( alpha=1e-4,
                    hidden_layer_sizes=(15,10,10,5, 2), random_state=1,verbose = True,max_iter=600)

clf.fit(X_train, y_train)                         

Iteration 1, loss = 0.65072336
Iteration 2, loss = 0.62263493
Iteration 3, loss = 0.56248207
Iteration 4, loss = 0.52489076
Iteration 5, loss = 0.51837287
Iteration 6, loss = 0.51590944
Iteration 7, loss = 0.51369019
Iteration 8, loss = 0.51228490
Iteration 9, loss = 0.51002084
Iteration 10, loss = 0.50865365
Iteration 11, loss = 0.50705294
Iteration 12, loss = 0.50590986
Iteration 13, loss = 0.50462219
Iteration 14, loss = 0.50371614
Iteration 15, loss = 0.50043818
Iteration 16, loss = 0.49850387
Iteration 17, loss = 0.49739621
Iteration 18, loss = 0.49664214
Iteration 19, loss = 0.48312993
Iteration 20, loss = 0.47849790
Iteration 21, loss = 0.47475515
Iteration 22, loss = 0.47375468
Iteration 23, loss = 0.47073472
Iteration 24, loss = 0.46779712
Iteration 25, loss = 0.46704880
Iteration 26, loss = 0.46458002
Iteration 27, loss = 0.46022775
Iteration 28, loss = 0.46973664
Iteration 29, loss = 0.47216413
Iteration 30, loss = 0.46912226
Iteration 31, loss = 0.46311110
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15, 10, 10, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=600, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [53]:
y_pred = clf.predict(X_test)

In [54]:
(y_pred==np.array(y_test)).mean()

0.7798138941144853

In [16]:
scipy.io.savemat('out_touchevent.mat', mdict={'X_train': X_train.values,
                                   'X_test':X_test.values,
                                   'Y_train':y_train,
                                    'Y_test': y_test})

In [ ]:
data = scipy.io.loadmat('out.mat')

In [ ]:
print _train.shape

data['Y_train'].shape

In [ ]:
## ====================   classifier =============

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1,1)
    def forward(self, x):
        x = self.fc1(x)
        return x

In [ ]:
import pymrmr